In [4]:
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
#import talos
import tensorflow as tf

#from tensorflow.keras.utils.layer_utils import count_params
from numpy.random import seed
from sklearn.model_selection import train_test_split
#from talos import Analyze, Deploy, Evaluate, Predict, Reporting, Restore, Scan
from tensorflow.keras.optimizers import SGD, Adadelta, Adam, Nadam, RMSprop
from tensorflow.keras.regularizers import l2

sys.path.append('/Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/')

print("Great")
from CelestialBodies.Planets import Earth
print("Even Better")
Earth()
from GravityModels.NN_Base import NN_Base
from GravityModels.NNSupport.NN_hyperparam import NN_hyperparam
from GravityModels.SphericalHarmonics import SphericalHarmonics
from GravNN.Preprocessors.MaxAbsTransform import MaxAbsTransform
from GravNN.Preprocessors.MinMaxTransform import MinMaxTransform
from GravNN.Preprocessors.RobustTransform import RobustTransform
from GravNN.Preprocessors.MinMaxStandardTransform  import MinMaxStandardTransform
from GravNN.Preprocessors.StandardTransform import StandardTransform
from GravNN.Support.transformations import (cart2sph,
                                     check_fix_radial_precision_errors,
                                     project_acceleration, sphere2cart)
from GravNN.Trajectories.DHGridDist import DHGridDist
from GravNN.Trajectories.RandomDist import RandomDist
from GravNN.Trajectories.UniformDist import UniformDist
from GravNN.Visualization.Grid import Grid
from GravNN.Visualization.MapVisualization import MapVisualization
from talos import Analyze
import pandas as pd

# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
# os.environ["RUNFILES_DIR"] = "/Users/johnmartin/Library/Python/3.7/share/plaidml"# plaidml might exist in different location. Look for "/usr/local/share/plaidml" and replace in above path
# os.environ["PLAIDML_NATIVE_PATH"] = "/Users/johnmartin/Library/Python/3.7/lib/libplaidml.dylib" # libplaidml.dylib might exist in different location. Look for "/usr/local/lib/libplaidml.dylib" and replace in above path

seed(1)

def plot_metrics(history, save_location=None):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    if save_location is not None:
        plt.savefig(plt.gcf(), save_location + "loss.pdf")
    return

def calc_metrics(meas, truth):
    error_inst = abs(np.divide((meas - truth),truth)*100)
    mse = np.square(meas - truth).mean(axis=None)
    rmse = np.sqrt(mse).mean(axis=None)
    print("Avg: " + str(np.average(error_inst)))
    print("Med: " + str(np.median(error_inst)))
    print("MSE: " + str(mse))
    print("RMSE: " + str(rmse))
    return error_inst, mse, rmse


def compute_error(model, 
                                    x_train_encode, y_train_encode,
                                    x_test_encode, y_test_encode
                                    , preprocessor):

    print("TRAINING ENCODE")
    pred_encode = model.predict(x_train_encode)
    error_inst, mse, rmse = calc_metrics(pred_encode, y_train_encode)
    
    print("TRAINING DECODE")
    prediction = model.predict(x_train_encode)
    x_decode, pred_decode = preprocessor.invert_transform(x_train_encode, prediction)
    x_decode, y_train_decode = preprocessor.invert_transform(x_train_encode, y_train_encode)
    error_inst, mse, rmse = calc_metrics(pred_decode, y_train_decode)

    print("TESTING ENCODE")
    prediction = model.predict(x_test_encode)
    error_inst, mse, rmse = calc_metrics(prediction, y_test_encode)
    
    print("TESTING DECODE")
    prediction = model.predict(x_test_encode)
    x_decode, pred_decode = preprocessor.invert_transform(x_test_encode, prediction)
    x_decode, y_test_decode = preprocessor.invert_transform(x_test_encode, y_test_encode)
    error_inst, mse, rmse = calc_metrics(pred_decode, y_test_decode)

    print("MAX DECODE PRED:" + str(pred_decode.max()))
    print("MAX DECODE TRUTH:"  + str(y_train_decode.max()))
    print("MIN DECODE PRED:"  + str(pred_decode.min()))
    print("MIN DECODE TRUTH:"  + str(y_train_decode.min()))
    return 



   

Great
Even Better


In [5]:


"""
Loads all parameters tested and find the one with the lowest validation loss
Populate a network with those parameters and retrain
"""
planet = Earth()

# directory = "./Hyperparams/UniformDist/MinMaxTransform/259200/"
# cases = ['071320150333.csv', '071320150753.csv', '071320150805.csv' , 
#                 '071420143549.csv', '071420143558.csv', '071420143613.csv']
# preprocessor = MinMaxTransform()
# trajectory = UniformDist(planet, planet.radius, point_count)

case = 6 # moved to main at bottom
directory = "./Hyperparams/RandomDist/MinMaxStandardTransform/259200/"
cases = ['072120093037.csv', '072120093056.csv', '072120093113.csv' , 
                '072120093251.csv', '072120093223.csv', '072120093140.csv']
preprocessor = MinMaxStandardTransform()

#point_count = 180*180*2
point_count = 259200 # 360*360*2
#point_count = 720*720*2
trajectory = RandomDist(planet, [planet.radius, planet.radius+5000.0], point_count)
#trajectory = RandomDist(planet, [planet.radius+330.0*1000-2500 , planet.radius + 330.0*1000+2500], point_count) #LEO
epochs = 30
bs = 50
plot_maps = True

experiment_dir = trajectory.trajectory_name + preprocessor.__class__.__name__
experiment_dir =  experiment_dir.replace(', ', '_')
experiment_dir =  experiment_dir.replace('[', '_')
experiment_dir =  experiment_dir.replace(']', '_')



save_location = "./Files/Final_NN/" + experiment_dir + "/case_"+str(case) + "/"
a = Analyze(directory + 'case_' + str(case) + '/' + cases[case-1]) 



df = a.data
run = df[df['val_loss'] == df['val_loss'].min()]
params = run.iloc[0].to_dict()
params['epochs'] = epochs
params['batch_size'] = 10#bs
params['optimizer'] = eval(params['optimizer'].split('.')[-1].split('\'')[0]) #Nadam
#params['kernel_initializer'] = eval(params['kernel_initializer'])
if eval(params['kernel_regularizer']) is None:
    params['kernel_regularizer'] = None
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', write_graph=True, write_images=True, histogram_freq=1)
params['callbacks'] = [tensorboard_callback]
print(params['callbacks'])
params['first_unit'] = 256
params['first_neuron'] = 128
params['hidden_layers'] = 10
params['activation'] = 'elu'
params['kernel_initializer'] = tf.keras.initializers.GlorotNormal()
'''
params['kernel_regularizer'] = 'l2'
params['first_unit'] = 256
params['first_neuron'] = 128
params['hidden_layers'] = 0
params['dropout'] = 0.4
params['batch_size'] = 10
params['lr'] = 0.1
params['epochs'] = 50
params['optimizer'] = Adadelta
params['losses'] = 'mean_absolute_error'
params['losses'] = 'mean_absolute_percentage_error'
params['optimizer'] = SGD
params['losses'] = 'mean_absolute_error'# 'mean_squared_error'
'''


sh_file = planet.sh_hf_file
max_deg = 1000

gravityModelMap = SphericalHarmonics(sh_file, degree=max_deg, trajectory=trajectory)
gravityModelMap.load() 
gravityModelMapC20 = SphericalHarmonics(sh_file, degree=2, trajectory=trajectory)
gravityModelMapC20.load() 
gravityModelMap.accelerations -= gravityModelMapC20.accelerations

pos_sphere = cart2sph(trajectory.positions)
pos_sphere = check_fix_radial_precision_errors(pos_sphere)
acc_proj = project_acceleration(pos_sphere, gravityModelMap.accelerations)

preprocessor.percentTest = 0.3
preprocessor.split(pos_sphere, acc_proj)
preprocessor.fit()
x_train, x_val, y_train, y_val = preprocessor.apply_transform()
#x_train, y_train = pos_sphere, acc_proj # don't even transform the data. 
# preprocessor = RobustTransform()
# a = Analyze("./Hyperparams/Initial_Search/Uniform/070820134910.csv") # Uniform Robust 1


hist, model = NN_hyperparam(x_train, y_train, x_val, y_val, params, verbose=1)# save_location=save_location)
#hist, model = NN_hyperparam(x_train, y_train, x_train, y_train, params, verbose=1) # Test on the same data

plot_metrics(hist)
compute_error(model, 
                            x_train, y_train,
                            x_val, y_val
                            , preprocessor)


The total number of points is not a perfect square
The total number of points changed to 259200
Found existing acceleration.data at /Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Trajectories/TrajectoryBase/../../Files/Trajectories/RandomDist/earthN_259200_RadBounds[6378136.6, 6383136.6]/SphericalHarmonics_EGM2008_to2190_TideFree_E_1000/
Found existing acceleration.data at /Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Trajectories/TrajectoryBase/../../Files/Trajectories/RandomDist/earthN_259200_RadBounds[6378136.6, 6383136.6]/SphericalHarmonics_EGM2008_to2190_TideFree_E_2/
Epoch 1/30
18144/18144 [==============================] - 41s 2ms/step - loss: 1.0002 - mse: 1.0002 - mae: 0.7015 - val_loss: 1.0038 - val_mse: 1.0038 - val_mae: 0.7022
Epoch 2/30
18144/18144 [==============================] - 44s 2ms/step - loss: 1.0002 - mse: 1.0002 - mae: 0.7015 - val_loss: 1.0036 - val_mse: 1.0036 - val_mae: 0.7023
Epoch 3/30
18144/18144 [=========

KeyboardInterrupt: 

In [2]:

map_grid = DHGridDist(planet, planet.radius, degree=175)
sh_all_gravityModel = SphericalHarmonics(sh_file, degree=max_deg, trajectory=map_grid)
sh_C20_gravityModel = SphericalHarmonics(sh_file, degree=2, trajectory=map_grid)
true_grid = Grid(gravityModel=sh_all_gravityModel)
sh_20_grid = Grid(gravityModel=sh_C20_gravityModel)
true_grid -= sh_20_grid #these values are projected
nn = NN_Base(model, preprocessor, test_traj=map_grid)

map_viz = MapVisualization(unit = 'mGal')
grid = Grid(gravityModel=nn, override=True)
#fig, ax = map_viz.plot_grid_mse(C100_grid, true_grid,vlim=[0, 2E-7])
map_viz.plot_grid(grid.total, "NN")
fig, ax = map_viz.plot_grid_rmse(grid, true_grid,vlim=[0, 40])

# M_params = count_params(nn.model.trainable_weights)
# print("Params: " + str(M_params))

plt.show()

NameError: name 'DHGridDist' is not defined